### Libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install srai[all]
!pip install contextily
!pip install alphashape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import zipfile
import srai
import os
from PIL import Image
import glob
import contextily as ctx
import alphashape


#To create embeddings
from srai.loaders import OSMOnlineLoader, OSMWayLoader, OSMPbfLoader
from srai.regionalizers import geocode_to_region_gdf, S2Regionalizer
from srai.plotting import plot_regions, plot_numeric_data
from srai.embedders import CountEmbedder, ContextualCountEmbedder,Hex2VecEmbedder, Highway2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
zip_path = '/content/drive/MyDrive/UC-TESIS/data/imagenes_stgo.zip'
extract_folder = '/content/drive/MyDrive/UC-TESIS/data'

In [ ]:
#with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#    zip_ref.extractall(extract_folder)

In [ ]:
image_folder = '/content/drive/MyDrive/UC-TESIS/data/imagenes_stgo'


In [ ]:
image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))]

coordinates = []
for image_file in image_files:
    try:
        # Extraer coordenadas del nombre del archivo
        coord_str = image_file.rsplit('.', 1)[0]
        lat, lon = map(float, coord_str.split(','))
        coordinates.append((lat, lon, image_file))
    except ValueError as e:
        print(f"Error procesando el archivo {image_file}: {e}")

if not coordinates:
    print("No se encontraron archivos con el formato esperado.")
else:
    print(f"Se encontraron {len(coordinates)} archivos con coordenadas.")


Error procesando el archivo -33.332885464,-70.527576184 (2).jpg: could not convert string to float: '-70.527576184 (2)'
Error procesando el archivo -33.472447335664334,-70.75639297857143 (2).jpg: could not convert string to float: '-70.75639297857143 (2)'
Error procesando el archivo -33.58075427972028,-70.671805 (2).jpg: could not convert string to float: '-70.671805 (2)'
Se encontraron 125377 archivos con coordenadas.


In [ ]:
image_files = [f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))]

for image_file in image_files[:10]:
    image_path = os.path.join(image_folder, image_file)
    print(image_path)
    image = Image.open(image_path)
    print(image)
    plt.imshow(image)
    plt.axis('off')  # No mostrar ejes
    plt.show()


#### Read coordinates from csv features file

In [6]:
df = pd.read_csv('/content/drive/MyDrive/UC-TESIS/data/vars_perceptuales_santiago.csv')

In [7]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf = gdf.set_crs(epsg=4326)
gdf

,latlong,beautiful,boring,depressing,lively,safe,wealthy,lat,lon,geometry
0,"-33.323944,-70.51263428391168",-0.306948,1.565049,0.572029,-1.137733,-0.120456,-0.561887,-33.323944,-70.512634,POINT (-70.51263 -33.32394)
1,"-33.323944,-70.5127291",-0.421388,0.309495,0.368965,-0.098733,-0.103042,-0.162294,-33.323944,-70.512729,POINT (-70.51273 -33.32394)
2,"-33.323944,-70.51298714285714",0.116505,0.164284,-0.110312,0.063860,0.391172,0.226372,-33.323944,-70.512987,POINT (-70.51299 -33.32394)
3,"-33.323944,-70.51343609999999",-0.159113,-0.500987,-0.213503,0.635165,0.300856,0.453708,-33.323944,-70.513436,POINT (-70.51344 -33.32394)
4,"-33.323944,-70.51379769565217",-1.226162,1.176751,1.462015,-0.842954,-0.946355,-0.936168,-33.323944,-70.513798,POINT (-70.51380 -33.32394)
...,...,...,...,...,...,...,...,...,...,...
121346,"-33.67884090851735,-70.68059514195582",-1.318599,2.201657,1.673377,-2.265229,-1.677807,-2.014101,-33.678841,-70.680595,POINT (-70.68060 -33.67884)
121347,"-33.67884090851735,-70.69912023659306",0.827014,1.129426,-0.431715,-1.053424,0.438107,0.078089,-33.678841,-70.699120,POINT (-70.69912 -33.67884)
121348,"-33.67884090851735,-70.70653027444796",0.975725,0.838188,-0.583064,-0.515637,0.957951,0.547197,-33.678841,-70.706530,POINT (-70.70653 -33.67884)
121349,"-33.67884090851735,-70.7176453312303",-0.629273,0.780489,0.881986,-1.363918,-1.328026,-1.726176,-33.678841,-70.717645,POINT (-70.71765 -33.67884)


In [ ]:

fig, ax = plt.subplots(figsize=(50,30))
ax.set_title('Photos location')
photo_points = gdf["geometry"].plot(ax=ax, marker="o", color="green", markersize=4)
# Add basemap
ctx.add_basemap(ax, crs=gdf.to_crs(4326).crs.to_string())

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
gdf

,latlong,beautiful,boring,depressing,lively,safe,wealthy,lat,lon,geometry
0,"-33.323944,-70.51263428391168",-0.306948,1.565049,0.572029,-1.137733,-0.120456,-0.561887,-33.323944,-70.512634,POINT (-70.51263 -33.32394)
1,"-33.323944,-70.5127291",-0.421388,0.309495,0.368965,-0.098733,-0.103042,-0.162294,-33.323944,-70.512729,POINT (-70.51273 -33.32394)
2,"-33.323944,-70.51298714285714",0.116505,0.164284,-0.110312,0.063860,0.391172,0.226372,-33.323944,-70.512987,POINT (-70.51299 -33.32394)
3,"-33.323944,-70.51343609999999",-0.159113,-0.500987,-0.213503,0.635165,0.300856,0.453708,-33.323944,-70.513436,POINT (-70.51344 -33.32394)
4,"-33.323944,-70.51379769565217",-1.226162,1.176751,1.462015,-0.842954,-0.946355,-0.936168,-33.323944,-70.513798,POINT (-70.51380 -33.32394)
...,...,...,...,...,...,...,...,...,...,...
121346,"-33.67884090851735,-70.68059514195582",-1.318599,2.201657,1.673377,-2.265229,-1.677807,-2.014101,-33.678841,-70.680595,POINT (-70.68060 -33.67884)
121347,"-33.67884090851735,-70.69912023659306",0.827014,1.129426,-0.431715,-1.053424,0.438107,0.078089,-33.678841,-70.699120,POINT (-70.69912 -33.67884)
121348,"-33.67884090851735,-70.70653027444796",0.975725,0.838188,-0.583064,-0.515637,0.957951,0.547197,-33.678841,-70.706530,POINT (-70.70653 -33.67884)
121349,"-33.67884090851735,-70.7176453312303",-0.629273,0.780489,0.881986,-1.363918,-1.328026,-1.726176,-33.678841,-70.717645,POINT (-70.71765 -33.67884)


Gdf with perceptual features

In [8]:
gdf_rm = gdf.copy().drop(columns=['lat', 'lon', 'latlong'])
gdf_rm['feature_id'] = range(1, len(gdf) + 1)
cols = gdf_rm.columns.tolist()
# Move 'unique_id' to the beginning
cols.insert(0, cols.pop(cols.index('feature_id')))
# Reorder the DataFrame columns
gdf_rm = gdf_rm[cols]
gdf_rm


,feature_id,beautiful,boring,depressing,lively,safe,wealthy,geometry
0,1,-0.306948,1.565049,0.572029,-1.137733,-0.120456,-0.561887,POINT (-70.51263 -33.32394)
1,2,-0.421388,0.309495,0.368965,-0.098733,-0.103042,-0.162294,POINT (-70.51273 -33.32394)
2,3,0.116505,0.164284,-0.110312,0.063860,0.391172,0.226372,POINT (-70.51299 -33.32394)
3,4,-0.159113,-0.500987,-0.213503,0.635165,0.300856,0.453708,POINT (-70.51344 -33.32394)
4,5,-1.226162,1.176751,1.462015,-0.842954,-0.946355,-0.936168,POINT (-70.51380 -33.32394)
...,...,...,...,...,...,...,...,...
121346,121347,-1.318599,2.201657,1.673377,-2.265229,-1.677807,-2.014101,POINT (-70.68060 -33.67884)
121347,121348,0.827014,1.129426,-0.431715,-1.053424,0.438107,0.078089,POINT (-70.69912 -33.67884)
121348,121349,0.975725,0.838188,-0.583064,-0.515637,0.957951,0.547197,POINT (-70.70653 -33.67884)
121349,121350,-0.629273,0.780489,0.881986,-1.363918,-1.328026,-1.726176,POINT (-70.71765 -33.67884)


#### Read files

In [ ]:
#gdf_rm.to_csv('/content/drive/MyDrive/UC-TESIS/data/gdf_rm_perceptual_features.csv', index=False)
#df = pd.read_csv('/content/drive/MyDrive/UC-TESIS/data/gdf_rm_perceptual_features.csv', index_col = False)
#df.drop(columns=['Unnamed: 0'], inplace=True)
#gdf_rm = gpd.GeoDataFrame(df, geometry= "geometry")
#gdf_rm = gdf_rm.set_crs(epsg=4326)

In [ ]:
gdf_rm

,feature_id,beautiful,boring,depressing,lively,safe,wealthy,geometry
0,1,-0.306948,1.565049,0.572029,-1.137733,-0.120456,-0.561887,POINT (-70.51263 -33.32394)
1,2,-0.421388,0.309495,0.368965,-0.098733,-0.103042,-0.162294,POINT (-70.51273 -33.32394)
2,3,0.116505,0.164284,-0.110312,0.063860,0.391172,0.226372,POINT (-70.51299 -33.32394)
3,4,-0.159113,-0.500987,-0.213503,0.635165,0.300856,0.453708,POINT (-70.51344 -33.32394)
4,5,-1.226162,1.176751,1.462015,-0.842954,-0.946355,-0.936168,POINT (-70.51380 -33.32394)
...,...,...,...,...,...,...,...,...
121346,121347,-1.318599,2.201657,1.673377,-2.265229,-1.677807,-2.014101,POINT (-70.68060 -33.67884)
121347,121348,0.827014,1.129426,-0.431715,-1.053424,0.438107,0.078089,POINT (-70.69912 -33.67884)
121348,121349,0.975725,0.838188,-0.583064,-0.515637,0.957951,0.547197,POINT (-70.70653 -33.67884)
121349,121350,-0.629273,0.780489,0.881986,-1.363918,-1.328026,-1.726176,POINT (-70.71765 -33.67884)


Create alphashape from photopoints

In [9]:
# Extraer los puntos de la columna 'geometry'
points = list(gdf_rm.geometry)
points = np.array([[point.x, point.y] for point in gdf_rm.geometry])
# Calcular el alpha shape
alpha = 100
alpha_shape = alphashape.alphashape(points, alpha)

# Crear un nuevo GeoDataFrame con el alpha shape
gdf_alpha_shape = gpd.GeoDataFrame(geometry=[alpha_shape])


In [ ]:
# Visualizar el resultado
fig, ax = plt.subplots(figsize=(50,30))
gdf.plot(ax=ax, color='green', marker='o', label='Puntos', markersize=4)
gdf_alpha_shape.boundary.plot(ax=ax, color='red', label='Alpha Shape', linewidth=10)
# Add basemap
ctx.add_basemap(ax, crs=gdf.to_crs(4326).crs.to_string())
plt.legend()
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
gdf_alpha_shape

,geometry
0,"MULTIPOLYGON (((-70.76705 -33.67659, -70.76581..."


Create new study area from shape

In [10]:
gdf_alpha_shape['region_id'] = "Santiago Metropolitan Region, Chile"
gdf_alpha_shape = gdf_alpha_shape.set_crs(epsg=4326)
study_area2 = gdf_alpha_shape
study_area2

,geometry,region_id
0,"MULTIPOLYGON (((-70.76705 -33.67659, -70.76581...","Santiago Metropolitan Region, Chile"


### Regionalizer RM

In [ ]:
study_area1 = geocode_to_region_gdf("MAIPU, Chile")
study_area2 = geocode_to_region_gdf("PUENTE ALTO, Chile")
study_area3 = geocode_to_region_gdf("LA FLORIDA, Chile")
study_area4 = geocode_to_region_gdf("SANTIAGO, Chile")
study_area5 = geocode_to_region_gdf("LAS CONDES, Chile")
study_area6 = geocode_to_region_gdf("SAN BERNARDO, Chile")
study_area7 = geocode_to_region_gdf("ÑUÑOA, Chile")
study_area8 = geocode_to_region_gdf("PUDAHUEL, Chile")
study_area9 = geocode_to_region_gdf("PROVIDENCIA, Chile")
study_area10 = geocode_to_region_gdf("PEÑALOLEN, Chile")
study_area11 = geocode_to_region_gdf("RECOLETA, Chile")
study_area12 = geocode_to_region_gdf("QUILICURA , Chile")
study_area13 = geocode_to_region_gdf("CONCHALI, Chile")
study_area14 = geocode_to_region_gdf("EL BOSQUE, Chile")
study_area15 = geocode_to_region_gdf("LA PINTANA, Chile")
study_area16 = geocode_to_region_gdf("CERRO NAVIA, Chile")
study_area17 = geocode_to_region_gdf("RENCA, Chile")
study_area18 = geocode_to_region_gdf("ESTACION CENTRAL, Chile")
study_area19 = geocode_to_region_gdf("QUINTA NORMAL, Chile")
study_area20 = geocode_to_region_gdf("LA REINA, Chile")
study_area21 = geocode_to_region_gdf("VITACURA, Chile")
study_area22 = geocode_to_region_gdf("SAN MIGUEL, Chile")
study_area23 = geocode_to_region_gdf("SAN JOAQUIN, Chile")
study_area24 = geocode_to_region_gdf("MACUL, Chile")
study_area25 = geocode_to_region_gdf("LO PRADO, Chile")
study_area26 = geocode_to_region_gdf("PEDRO AGUIRRE CERDA, Chile")
study_area27 = geocode_to_region_gdf("SAN RAMON, Chile")
study_area28 = geocode_to_region_gdf("LA CISTERNA, Chile")
study_area29 = geocode_to_region_gdf("LA GRANJA, Chile")
#study_area30 = geocode_to_region_gdf("LO BARNECHEA, Chile")
study_area31 = geocode_to_region_gdf("HUECHURABA, Chile")
study_area32 = geocode_to_region_gdf("CERRILLOS, Chile")
study_area33 = geocode_to_region_gdf("INDEPENDENCIA, Chile")
study_area34 = geocode_to_region_gdf("LO ESPEJO, Chile")
# Concatenar todos los GeoDataFrames
study_area = [
    study_area1, study_area2, study_area3, study_area4, study_area5, study_area6,
    study_area7, study_area8, study_area9, study_area10, study_area11, study_area12,
    study_area13, study_area14, study_area15, study_area16, study_area17, study_area18,
    study_area19, study_area20, study_area21, study_area22, study_area23, study_area24,
    study_area25, study_area26, study_area27, study_area28, study_area29,
    study_area31, study_area32, study_area33, study_area34
] # study_area30,

study_area = gpd.GeoDataFrame(pd.concat(study_area, ignore_index=True))
study_area = study_area.reset_index(drop=True)
study_area['region_id'] = "Santiago Metropolitan Region, Chile"
study_area

In [ ]:
regionalizer = H3Regionalizer(resolution=7, buffer=True)
regions_gdf_rm_7 = regionalizer.transform(study_area)
plot_regions(regions_gdf_rm_7, tiles_style="CartoDB positron")

In [ ]:
regions_gdf_rm_7

,geometry
region_id,
87b2c5094ffffff,"POLYGON ((-70.60124 -33.47777, -70.60943 -33.4..."
87b2c508bffffff,"POLYGON ((-70.49903 -33.54541, -70.50722 -33.5..."
87b2c551effffff,"POLYGON ((-70.87584 -33.39274, -70.88406 -33.4..."
87b2c546dffffff,"POLYGON ((-70.60689 -33.53745, -70.61508 -33.5..."
87b2c50b6ffffff,"POLYGON ((-70.59747 -33.43796, -70.60566 -33.4..."
...,...
87b2c555affffff,"POLYGON ((-70.73187 -33.48000, -70.74008 -33.4..."
87b2c544dffffff,"POLYGON ((-70.63337 -33.58753, -70.64157 -33.5..."
87b2c50b1ffffff,"POLYGON ((-70.55584 -33.45710, -70.56403 -33.4..."


In [ ]:
regions_gdf_rm_7.shape

(333, 1)

In [ ]:
regionalizer = H3Regionalizer(resolution=8, buffer=True)
regions_gdf_rm_8 = regionalizer.transform(study_area)
plot_regions(regions_gdf_rm_8, tiles_style="CartoDB positron")

In [ ]:
regions_gdf_rm_8.shape

(2098, 1)

In [ ]:
regionalizer = H3Regionalizer(resolution=8, buffer=True)
regions_gdf_rm_8 = regionalizer.transform(study_area2)
plot_regions(regions_gdf_rm_8, tiles_style="CartoDB positron")

In [ ]:
regions_gdf_rm_8.shape

(1925, 1)

#### R = 9

In [ ]:
regionalizer = H3Regionalizer(resolution=9, buffer=True)
regions_gdf_rm_9 = regionalizer.transform(study_area)
plot_regions(regions_gdf_rm_9, tiles_style="CartoDB positron")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
regions_gdf_rm_9.shape

(14079, 1)

##### Alphashape region

In [11]:
regionalizer = H3Regionalizer(resolution=9, buffer=True)
regions_gdf_rm_9 = regionalizer.transform(study_area2)
plot_regions(regions_gdf_rm_9, tiles_style="CartoDB positron")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
regions_gdf_rm_9.shape

(12734, 1)

#### R=10

In [ ]:
regionalizer = H3Regionalizer(resolution=10, buffer=True)
regions_gdf_rm_10 = regionalizer.transform(study_area)
plot_regions(regions_gdf_rm_10, tiles_style="CartoDB positron")

In [ ]:
regions_gdf_rm_10.shape

#### Alphashape region

In [12]:
regionalizer = H3Regionalizer(resolution=10, buffer=True)
regions_gdf_rm_10 = regionalizer.transform(study_area2)
plot_regions(regions_gdf_rm_10, tiles_style="CartoDB positron")

In [23]:
regions_gdf_rm_10.shape

### **Joiner**

Resolution = 7

* Total regions = 585
* Total regions wth at least 1 feature = 263
* Number of regions with 1 or more features: 263
* Number of regions with 3 or more features: 260
* Number of regions with 5 or more features: 257
* Number of regions with 8 or more features: 252
* Number of regions with 13 or more features: 247

In [ ]:
#OSM+REGIONS
joiner = IntersectionJoiner()
joint_rm_gdf_photos_7 = joiner.transform(
    regions_gdf_rm_7.set_crs(epsg=4326, inplace=True),
    gdf_rm.set_crs(epsg=4326, inplace=True),
    return_geom=True)

joint_rm_gdf_photos_7

geometry
region_id       feature_id                             
87b2c5cdeffffff 108         POINT (-70.70484 -33.32394)
                109         POINT (-70.70524 -33.32394)
                110         POINT (-70.70530 -33.32394)
                111         POINT (-70.70574 -33.32394)
                112         POINT (-70.70589 -33.32394)
...                                                 ...
87b2c57b2ffffff 121313      POINT (-70.76458 -33.67659)
                121314      POINT (-70.76581 -33.67659)
                121315      POINT (-70.76705 -33.67659)
                121316      POINT (-70.76828 -33.67659)
                121319      POINT (-70.76816 -33.67712)

[115069 rows x 1 columns]

In [ ]:
joint_rm_gdf_photos_7_sep= joint_rm_gdf_photos_7.reset_index()

In [ ]:
region_id_counts = joint_rm_gdf_photos_7_sep['region_id'].value_counts()
print("\nRegion ID Counts:\n", region_id_counts)


Region ID Counts:
 region_id
87b2c5466ffffff    995
87b2c546effffff    984
87b2c5555ffffff    979
87b2c546cffffff    975
87b2c5562ffffff    973
                  ... 
87b2c5451ffffff      3
87b2c518bffffff      3
87b2c50aeffffff      2
87b2c551bffffff      1
87b2c5088ffffff      1
Name: count, Length: 250, dtype: int64


In [ ]:
feature_counts = joint_rm_gdf_photos_7_sep.groupby('region_id')['feature_id'].count()
feature_counts

region_id
87b2c5012ffffff     12
87b2c5080ffffff    319
87b2c5081ffffff     39
87b2c5082ffffff    446
87b2c5083ffffff    388
                  ... 
87b2c5cd9ffffff    401
87b2c5cdaffffff    378
87b2c5cdbffffff    456
87b2c5cddffffff     17
87b2c5cdeffffff    385
Name: feature_id, Length: 250, dtype: int64

In [ ]:
# Filter feature counts to find regions with 3 or more features
regions_with_0 = feature_counts[feature_counts == 0]
regions_with_1_or_more = feature_counts[feature_counts >= 1]
regions_with_3_or_more = feature_counts[feature_counts >= 3]
regions_with_5_or_more = feature_counts[feature_counts >= 5]
regions_with_8_or_more = feature_counts[feature_counts >= 8]
regions_with_13_or_more = feature_counts[feature_counts >= 13]
print("Number of regions with 0 features:", len(regions_with_0))
print("Number of regions with 1 or more features:", len(regions_with_1_or_more))
print("Number of regions with 3 or more features:", len(regions_with_3_or_more))
print("Number of regions with 5 or more features:", len(regions_with_5_or_more))
print("Number of regions with 8 or more features:", len(regions_with_8_or_more))
print("Number of regions with 13 or more features:", len(regions_with_13_or_more))

# region_id and their respective feature counts
print("\nRegions with 3 or more features:")
print(regions_with_3_or_more)

Number of regions with 0 features: 0
Number of regions with 1 or more features: 250
Number of regions with 3 or more features: 247
Number of regions with 5 or more features: 244
Number of regions with 8 or more features: 241
Number of regions with 13 or more features: 237

Regions with 3 or more features:
region_id
87b2c5012ffffff     12
87b2c5080ffffff    319
87b2c5081ffffff     39
87b2c5082ffffff    446
87b2c5083ffffff    388
                  ... 
87b2c5cd9ffffff    401
87b2c5cdaffffff    378
87b2c5cdbffffff    456
87b2c5cddffffff     17
87b2c5cdeffffff    385
Name: feature_id, Length: 247, dtype: int64


**Resolution = 8**

* Total regions = 3.765
* Total regions wth at least 1 feature = 1.479
* Number of regions with 1 or more features: 1479
* Number of regions with 3 or more features: 1423
* Number of regions with 5 or more features: 1391
* Number of regions with 8 or more features: 1357
* Number of regions with 13 or more features: 1300

In [ ]:
#OSM+REGIONS
joiner = IntersectionJoiner()
joint_rm_gdf_photos_8 = joiner.transform(
    regions_gdf_rm_8.set_crs(epsg=4326, inplace=True),
    gdf_rm.set_crs(epsg=4326, inplace=True),
    return_geom=True)

joint_rm_gdf_photos_8

geometry
region_id       feature_id                             
88b2c5cde9fffff 108         POINT (-70.70484 -33.32394)
                109         POINT (-70.70524 -33.32394)
                110         POINT (-70.70530 -33.32394)
                111         POINT (-70.70574 -33.32394)
                112         POINT (-70.70589 -33.32394)
...                                                 ...
88b2c57b21fffff 121313      POINT (-70.76458 -33.67659)
                121314      POINT (-70.76581 -33.67659)
                121315      POINT (-70.76705 -33.67659)
                121316      POINT (-70.76828 -33.67659)
                121319      POINT (-70.76816 -33.67712)

[113561 rows x 1 columns]

In [ ]:
joint_rm_gdf_photos_8_sep= joint_rm_gdf_photos_8.reset_index()
region_id_counts = joint_rm_gdf_photos_8_sep['region_id'].value_counts()
print("\nRegion ID Counts:\n", region_id_counts)


Region ID Counts:
 region_id
88b2c54611fffff    151
88b2c54431fffff    150
88b2c546cbfffff    149
88b2c555e3fffff    149
88b2c50937fffff    149
                  ... 
88b2c552abfffff      1
88b2c518c9fffff      1
88b2c518a3fffff      1
88b2c55709fffff      1
88b2c5085dfffff      1
Name: count, Length: 1414, dtype: int64


In [ ]:
feature_counts = joint_rm_gdf_photos_8_sep.groupby('region_id')['feature_id'].count()
feature_counts

region_id
88b2c50121fffff     4
88b2c5012dfffff    11
88b2c50801fffff    63
88b2c50803fffff    94
88b2c50805fffff    42
                   ..
88b2c5cde5fffff    74
88b2c5cde7fffff    66
88b2c5cde9fffff    19
88b2c5cdebfffff    61
88b2c5cdedfffff    53
Name: feature_id, Length: 1414, dtype: int64

In [ ]:
# Filter feature counts to find regions with 3 or more features
regions_with_1_or_more = feature_counts[feature_counts >= 1]
regions_with_3_or_more = feature_counts[feature_counts >= 3]
regions_with_5_or_more = feature_counts[feature_counts >= 5]
regions_with_8_or_more = feature_counts[feature_counts >= 8]
regions_with_13_or_more = feature_counts[feature_counts >= 13]
print("Number of regions with 1 or more features:", len(regions_with_1_or_more))
print("Number of regions with 3 or more features:", len(regions_with_3_or_more))
print("Number of regions with 5 or more features:", len(regions_with_5_or_more))
print("Number of regions with 8 or more features:", len(regions_with_8_or_more))
print("Number of regions with 13 or more features:",len(regions_with_13_or_more))

# region_id and their respective feature counts
print("\nRegions with 3 or more features:")
print(regions_with_8_or_more)

Number of regions with 1 or more features: 1414
Number of regions with 3 or more features: 1363
Number of regions with 5 or more features: 1334
Number of regions with 8 or more features: 1304
Number of regions with 13 or more features: 1253

Regions with 3 or more features:
region_id
88b2c5012dfffff    11
88b2c50801fffff    63
88b2c50803fffff    94
88b2c50805fffff    42
88b2c50807fffff    27
                   ..
88b2c5cde5fffff    74
88b2c5cde7fffff    66
88b2c5cde9fffff    19
88b2c5cdebfffff    61
88b2c5cdedfffff    53
Name: feature_id, Length: 1304, dtype: int64


#### R=9

**Resolution = 9**

* Total regions = 25.479
* Total regions with at least 1 feature = 8.209
* Number of regions with 1 or more features: 8209
* Number of regions with 3 or more features: 7703
* Number of regions with 5 or more features: 7219
* Number of regions with 8 or more features: 6497
* Number of regions with 13 or more features: 5199

**Resolution = 9 alphashape**

* Total regions = 12.734
* Total regions with at least 1 feature = 9.108
* Number of regions with 1 or more features: 9108
* Number of regions with 3 or more features: 8363
* Number of regions with 5 or more features: 7706
* Number of regions with 8 or more features: 6755
* Number of regions with 13 or more features: 5284

In [ ]:
#OSM+REGIONS
joiner = IntersectionJoiner()
joint_rm_gdf_photos_9 = joiner.transform(
    regions_gdf_rm_9.set_crs(epsg=4326, inplace=True),
    gdf_rm.set_crs(epsg=4326, inplace=True),
    return_geom=True)

joint_rm_gdf_photos_9

Number of features in each region

In [ ]:
joint_rm_gdf_photos_9_sep= joint_rm_gdf_photos_9.reset_index()
region_id_counts = joint_rm_gdf_photos_9_sep['region_id'].value_counts()
feature_counts = joint_rm_gdf_photos_9_sep.groupby('region_id')['feature_id'].count()
print("\nRegion ID Counts:\n", region_id_counts)
print("\Feature ID Counts:\n", feature_counts)

Regiones con al menos 1,3,5,8,13 photos

In [2]:
# Filter feature counts to find regions with 3 or more features
regions_with_1_or_more = feature_counts[feature_counts >= 1]
regions_with_3_or_more = feature_counts[feature_counts >= 3]
regions_with_5_or_more = feature_counts[feature_counts >= 5]
regions_with_8_or_more = feature_counts[feature_counts >= 8]
regions_with_13_or_more = feature_counts[feature_counts >= 13]
print("Number of regions with 1 or more features:", len(regions_with_1_or_more))
print("Number of regions with 3 or more features:", len(regions_with_3_or_more))
print("Number of regions with 5 or more features:", len(regions_with_5_or_more))
print("Number of regions with 8 or more features:", len(regions_with_8_or_more))
print("Number of regions with 13 or more features:", len(regions_with_13_or_more))

NameError: name 'feature_counts' is not defined

### Maps with regions

**Resolution = 9**

* Average Hexagon Area (km2):

0.10533251
(105,332.5m2)

* Average edge length (Km):

0.20078


In [1]:
selected_region_ids_1 = regions_with_1_or_more.index.tolist()
selected_region_ids_3 = regions_with_3_or_more.index.tolist()
selected_region_ids_5 = regions_with_5_or_more.index.tolist()
selected_region_ids_8 = regions_with_8_or_more.index.tolist()
selected_region_ids_13 =regions_with_13_or_more.index.tolist()

NameError: name 'regions_with_1_or_more' is not defined

In [ ]:
# Filter the original GeoDataFrame to include only selected regions
selected_region_ids_3 = regions_with_3_or_more.index.tolist()
selected_region_ids_3
filtered_gdf = joint_rm_gdf_photos_9_sep[joint_rm_gdf_photos_9_sep['region_id'].isin(selected_region_ids_3)]
filtered_gdf

,region_id,feature_id,geometry
0,89b2c51a22bffff,0,POINT (-70.51263 -33.32394)
1,89b2c51a22bffff,1,POINT (-70.51273 -33.32394)
2,89b2c51a22bffff,2,POINT (-70.51299 -33.32394)
3,89b2c51a22bffff,238,POINT (-70.51147 -33.32506)
4,89b2c51a22bffff,239,POINT (-70.51273 -33.32506)
...,...,...,...
121326,89b2c57b21bffff,121316,POINT (-70.76828 -33.67659)
121327,89b2c57b21bffff,121319,POINT (-70.76816 -33.67712)
121328,89b2c578493ffff,121322,POINT (-70.69820 -33.67740)
121329,89b2c578493ffff,121325,POINT (-70.69741 -33.67770)


In [ ]:
plot_regions(filtered_gdf, tiles_style="CartoDB positron")


In [ ]:
# Select unique regions
unique_regions = filtered_gdf.drop_duplicates(subset=['region_id'])

# Plot the unique regions
plot_regions(unique_regions, tiles_style="CartoDB positron")

Output hidden; open in https://colab.research.google.com to view.

#### R=10

**Resolution = 10**

Total regions = 44.727

* Number of regions with 1 or more features: 44727
* Number of regions with 3 or more features: 23462
* Number of regions with 5 or more features: 1783
* Number of regions with 8 or more features: 0
* Number of regions with 13 or more features: 0

**Resolution = 10 alphashape**

 Total regions = 87.110
* Number of regions with 1 or more features: 47588
* Number of regions with 3 or more features: 24220
* Number of regions with 5 or more features: 1831
* Number of regions with 8 or more features: 0
* Number of regions with 13 or more features: 0

In [ ]:
#OSM+REGIONS r=10
joiner = IntersectionJoiner()
joint_rm_gdf_photos_10 = joiner.transform(
    regions_gdf_rm_10.set_crs(epsg=4326, inplace=True),
    gdf_rm.set_crs(epsg=4326, inplace=True),
    return_geom=True)

joint_rm_gdf_photos_10


geometry
region_id       feature_id                             
8ab2c51a2297fff 0           POINT (-70.51263 -33.32394)
                1           POINT (-70.51273 -33.32394)
8ab2c51a274ffff 2           POINT (-70.51299 -33.32394)
                3           POINT (-70.51344 -33.32394)
                4           POINT (-70.51380 -33.32394)
...                                                 ...
8ab2c57a30affff 121346      POINT (-70.68060 -33.67884)
8ab2c57849affff 121347      POINT (-70.69912 -33.67884)
8ab2c5784d37fff 121348      POINT (-70.70653 -33.67884)
8ab2c57b1067fff 121349      POINT (-70.71765 -33.67884)
8ab2c57b10a7fff 121350      POINT (-70.72506 -33.67884)

[121345 rows x 1 columns]

In [ ]:
joint_rm_gdf_photos_10_sep= joint_rm_gdf_photos_10.reset_index()
region_id_counts = joint_rm_gdf_photos_10_sep['region_id'].value_counts()
print("\nRegion ID Counts:\n", region_id_counts)


Region ID Counts:
 region_id
8ab2c5575b9ffff    7
8ab2c5561927fff    7
8ab2c50a255ffff    7
8ab2c5445087fff    7
8ab2c5554597fff    7
                  ..
8ab2c5545287fff    1
8ab2c540b55ffff    1
8ab2c54e490ffff    1
8ab2c556a10ffff    1
8ab2c57b10a7fff    1
Name: count, Length: 47588, dtype: int64


In [ ]:
feature_counts = joint_rm_gdf_photos_10_sep.groupby('region_id')['feature_id'].count()
feature_counts

region_id
8ab2c5012127fff    1
8ab2c5012137fff    1
8ab2c5012c4ffff    4
8ab2c5012c8ffff    2
8ab2c5012caffff    1
                  ..
8ab2c5cded5ffff    2
8ab2c5cded87fff    2
8ab2c5cded8ffff    2
8ab2c5cded97fff    2
8ab2c5cded9ffff    2
Name: feature_id, Length: 47588, dtype: int64

In [ ]:
# Filter feature counts to find regions with 3 or more features
regions_with_1_or_more = feature_counts[feature_counts >= 1]
regions_with_3_or_more = feature_counts[feature_counts >= 3]
regions_with_5_or_more = feature_counts[feature_counts >= 5]
regions_with_8_or_more = feature_counts[feature_counts >= 8]
regions_with_13_or_more = feature_counts[feature_counts >= 13]
print("Number of regions with 1 or more features:", len(regions_with_1_or_more))
print("Number of regions with 3 or more features:", len(regions_with_3_or_more))
print("Number of regions with 5 or more features:", len(regions_with_5_or_more))
print("Number of regions with 8 or more features:", len(regions_with_8_or_more))
print("Number of regions with 13 or more features:", len(regions_with_13_or_more))

# region_id and their respective feature counts
print("\nRegions with 13 or more features:")
print(regions_with_13_or_more)

Number of regions with 1 or more features: 47588
Number of regions with 3 or more features: 24220
Number of regions with 5 or more features: 1831
Number of regions with 8 or more features: 0
Number of regions with 13 or more features: 0

Regions with 13 or more features:
Series([], Name: feature_id, dtype: int64)


### Maps with regions

**Resolution = 10**

* Average Hexagon Area (km2):
0.0150
(15,047.5m2)

* Average edge length (Km):
0.0758



### Hex2vec Embedder

In [ ]:
neighbourhood = H3Neighbourhood(regions_gdf_rm_9)
embedder = Hex2VecEmbedder([150, 75, 25])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    h3_embeddings_r9_all = embedder.fit_transform(
        regions_gdf_rm_9,
        perceptual_features_gdf_rm,
        joint_rm_gdf_photos_9,
        neighbourhood,
        #trainer_kwargs={"max_epochs":10, "accelerator": "gpu"},
        batch_size=128)

h3_embeddings_r9_all

100%|██████████| 14079/14079 [00:00<00:00, 23573.50it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 14.3 K | train
-----------------------------------------------
14.3 K    Trainable params
0         Non-trainable params
14.3 K    Total params
0.057     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
region_id,,,,,,,,,,,,,,,,,,,,,
89b2c555cc3ffff,-0.204542,0.425151,-0.082246,0.235761,-0.345377,-0.236822,0.101078,-0.234366,-0.080589,-0.050214,...,0.204649,-0.150341,-0.208449,-0.547481,0.116267,0.004879,-0.470202,0.332491,0.302315,-0.103883
89b2c5083b7ffff,-0.021552,0.012639,0.077991,-0.233882,0.275353,0.043776,0.300034,-0.050290,0.072021,0.087550,...,-0.058233,-0.185394,0.136059,0.124900,-0.030709,0.333858,0.209837,0.096745,-0.359900,-0.219542
89b2c54341bffff,-0.021552,0.012639,0.077991,-0.233882,0.275353,0.043776,0.300034,-0.050290,0.072021,0.087550,...,-0.058233,-0.185394,0.136059,0.124900,-0.030709,0.333858,0.209837,0.096745,-0.359900,-0.219542
89b2c55639bffff,0.029926,0.117670,0.095504,0.151756,-0.306475,-0.234100,-0.389339,0.108929,0.118520,-0.191143,...,0.122130,0.077734,-0.298639,0.055627,-0.093999,-0.243877,-0.468588,-0.172134,0.037197,0.198336
89b2c54514fffff,-0.021552,0.012639,0.077991,-0.233882,0.275353,0.043776,0.300034,-0.050290,0.072021,0.087550,...,-0.058233,-0.185394,0.136059,0.124900,-0.030709,0.333858,0.209837,0.096745,-0.359900,-0.219542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89b2c509a73ffff,0.254610,-0.002320,0.060407,0.104945,0.030921,-0.339270,-0.094113,-0.175662,0.017590,-0.322480,...,0.007851,0.366623,-0.483370,-0.231182,-0.085371,-0.158892,0.083316,0.035750,0.332994,0.234960
89b2c5416c3ffff,-0.021552,0.012639,0.077991,-0.233882,0.275353,0.043776,0.300034,-0.050290,0.072021,0.087550,...,-0.058233,-0.185394,0.136059,0.124900,-0.030709,0.333858,0.209837,0.096745,-0.359900,-0.219542
89b2c571613ffff,0.493284,0.236797,0.072399,0.035330,0.234789,0.064018,-0.125369,-0.189769,-0.346892,-0.303721,...,-0.301039,0.467097,-0.117520,-0.191820,0.435533,0.052301,0.483122,0.133643,0.375683,0.346992


In [ ]:
folium_map = plot_regions(study_area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions_gdf_rm_9, 0, h3_embeddings_r9_all, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

CountEmbedder

In [ ]:

embedder = CountEmbedder()
h3_count_embed_r9_all = embedder.transform(
        regions_gdf_rm_9,
        perceptual_features_gdf_rm,
        joint_rm_gdf_photos_9)

h3_count_embed_r9_all

,beautiful,boring,depressing,lively,safe,wealthy
region_id,,,,,,
89b2c509227ffff,1,4,0,-5,0,0
89b2c552ea7ffff,0,0,0,0,0,0
89b2c542293ffff,0,0,0,0,0,0
89b2c5090b7ffff,-5,-2,3,0,-7,-6
89b2c5d4923ffff,0,0,0,0,0,0
...,...,...,...,...,...,...
89b2c588b37ffff,0,0,0,0,0,0
89b2c589e9bffff,0,0,0,0,0,0
89b2c5133a7ffff,0,0,0,0,0,0


R=9 all photos

In [ ]:
folium_map = plot_regions(study_area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions_gdf_rm_9, "beautiful", h3_count_embed_r9_all, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#study_area['region_id'] = "Santiago Metropolitan Region, Chile"

### R=10
Hex2vec

In [ ]:
neighbourhood = H3Neighbourhood(regions_gdf_rm_10)
embedder = Hex2VecEmbedder([150, 75, 25])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    h3_embeddings_r10_all = embedder.fit_transform(
        regions_gdf_rm_10,
        perceptual_features_gdf_rm,
        joint_rm_gdf_photos_10,
        neighbourhood,
        trainer_kwargs={"max_epochs":10, "accelerator": "gpu"},
        batch_size=128)

h3_embeddings_r10_all

100%|██████████| 175972/175972 [00:07<00:00, 24328.09it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 14.3 K | train
-----------------------------------------------
14.3 K    Trainable params
0         Non-trainable params
14.3 K    Total params
0.057     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
region_id,,,,,,,,,,,,,,,,,,,,,
8ab2c544e14ffff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
8ab2c0b6189ffff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
8ab2c54152a7fff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
8ab2c554d8a7fff,0.167551,0.237176,0.305832,-0.022579,-0.048652,-0.002696,-1.356942,0.080421,0.190231,0.000586,...,0.015146,-0.022872,-0.095966,-0.323581,0.172664,-0.680832,0.991657,0.612795,-0.012270,0.025738
8ab2c5026097fff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8ab2c5c68757fff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
8ab2c511c30ffff,-0.010889,-0.204032,-0.013219,-0.007083,-0.014742,0.007214,0.229250,-0.006258,-0.018116,0.232241,...,-0.029390,-0.004827,-0.073494,0.097947,-0.013611,-0.057198,-0.089883,-0.071834,0.031381,-0.001486
8ab2c555188ffff,-0.043581,0.504993,-0.090210,0.043691,-0.113093,0.016140,-0.257830,-0.023960,-0.052331,-0.833353,...,0.067022,0.012193,0.374465,-0.148044,-0.111835,0.374228,-0.468483,-0.165881,0.055361,-0.013985


In [ ]:
folium_map = plot_regions(study_area, colormap='BuGn', tiles_style="CartoDB positron")
plot_numeric_data(regions_gdf_rm_10, 0, h3_embeddings_r9_all, map=folium_map)

IndexError: list index out of range

CountEmbedder

In [ ]:

embedder = CountEmbedder()
h3_count_embed_r10_all = embedder.transform(
        regions_gdf_rm_10,
        perceptual_features_gdf_rm,
        joint_rm_gdf_photos_10)

h3_count_embed_r10_all

,beautiful,boring,depressing,lively,safe,wealthy
region_id,,,,,,
8ab2c544e14ffff,0,0,0,0,0,0
8ab2c0b6189ffff,0,0,0,0,0,0
8ab2c54152a7fff,0,0,0,0,0,0
8ab2c554d8a7fff,2,-4,-3,5,4,4
8ab2c5026097fff,0,0,0,0,0,0
...,...,...,...,...,...,...
8ab2c5c68757fff,0,0,0,0,0,0
8ab2c511c30ffff,0,0,0,0,0,0
8ab2c555188ffff,-2,1,2,-2,-2,-2


In [ ]:
folium_map = plot_regions(study_area, colormap=['BuGn'], tiles_style="CartoDB positron")
plot_numeric_data(regions_gdf_rm_10, "beautiful", h3_count_embed_r10_all, map=folium_map)

In [ ]:
display(folium_map)

In [ ]:

folium_map = plot_regions(study_area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")

# Iterate over columns of the DataFrame and plot
for column in h3_count_embed_r10_all.columns:
    plot_numeric_data(regions_gdf_rm_10, column, h3_count_embed_r10_all, map=folium_map)

display(folium_map)